In [52]:
import pandas as pd
import zipfile

df = pd.read_csv(zipfile.ZipFile('sentiAnalysis.zip').open('sentiAnalysis.csv'))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [15]:
df

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,Duration_hours,label,Pos_Score,Neg_Score,emotion,Emotionality,eventual
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- setup project contain * .gif resource ; - re...,CLOSED,FIXED,2.0,2001-10-11 01:34:00+00:00,2012-02-09 20:57:47+00:00,90571.383333,long,0.051768,0.065657,negative,0.117424,FIXED
1,2,P5,Team,NaN,Opening repository resources doesnt honor type...,open repository resource always open default t...,RESOLVED,FIXED,2.0,2001-10-11 01:34:00+00:00,2002-05-07 14:33:56+00:00,5004.983333,short,0.020833,0.056818,negative,0.077652,FIXED
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),km ( 10/2/2001 5:55:18 pm ) ; pr deletion indi...,RESOLVED,FIXED,2.0,2001-10-11 01:34:00+00:00,2010-05-07 14:28:53+00:00,75132.900000,long,0.086364,0.036364,positive,0.122727,FIXED
3,4,P5,Team,NaN,need better error message if catching up over ...,- become synchronize project repository ; - us...,RESOLVED,FIXED,2.0,2001-10-11 01:34:00+00:00,2002-03-01 21:27:31+00:00,3403.883333,short,0.049342,0.092105,negative,0.141447,FIXED
4,5,P3,Team,NaN,ISharingManager sharing API inconsistent (1GAU...,getting/setting manage state resource ; method...,RESOLVED,WONTFIX,2.0,2001-10-11 01:34:00+00:00,2008-08-15 12:04:36+00:00,60010.500000,long,0.085227,0.005682,positive,0.090909,NOTFIXED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68119,229777,P3,UI,NaN,[Viewers] Wrong argument in the first statemen...,build id : 3.3 ( i sure mean build id ; 3.3 ec...,RESOLVED,FIXED,3.3,2008-05-01 13:47:00+00:00,2008-05-10 14:06:07+00:00,216.316667,short,0.064024,0.027439,positive,0.091463,FIXED
68120,229779,P3,UI,NaN,NPE in performance tests,several npes within ui session test prevent ru...,VERIFIED,FIXED,3.4,2008-05-01 13:52:00+00:00,2008-05-20 14:12:14+00:00,456.333333,short,0.033333,0.033333,negative,0.066667,FIXED
68121,229782,P3,UI,NaN,Performance tests for ICU Collator,i20080501-0100 ; ; use collator ( see dependan...,VERIFIED,FIXED,3.4,2008-05-01 14:05:00+00:00,2009-06-01 18:25:12+00:00,9508.333333,long,0.025000,0.000000,positive,0.025000,FIXED
68122,229789,P3,UI,NaN,[Examples] examples plugins create duplicate m...,create attachment 98318 ; screenshot ; ; i2008...,VERIFIED,FIXED,3.4,2008-05-01 15:02:00+00:00,2008-05-31 01:57:57+00:00,706.916667,short,0.092105,0.013158,positive,0.105263,FIXED


In [55]:
def categorize_label(label):
    if 'FIXED' in label.upper():  # Convert to uppercase and check for 'FIXED'
        return 'FIXED'
    else:
        return 'NOTFIXED'

df['eventual'] = df['Resolution'].apply(categorize_label)

In [56]:
def train_naive_bayes(df, feature_cols, target_col, test_size=0.2, random_state=42):
    """
    Trains a Naive Bayes model on the specified DataFrame using the given feature columns and target column.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    feature_cols (list of str): List of column names to be used as features.
    target_col (str): The name of the target column.
    test_size (float): The proportion of the dataset to include in the test split.
    random_state (int): The seed used by the random number generator.

    Returns:
    model: Trained Naive Bayes model.
    accuracy: Accuracy of the model on the test set.
    report: Classification report of the model on the test set.
    """
    if 'label' in target_col:
        df = df[df['Resolution'].str.contains('FIXED', na=False)]

    
    # Fill missing values in feature columns
    df[feature_cols] = df[feature_cols].fillna(' ')

    # Combine feature columns into a single column for vectorization
    df['combined'] = df[feature_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

    # Encode the combined text column
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['combined'])

    # Encode the target column
    y = df[target_col]

    # Ensure X and y have the same length
    assert X.shape[0] == len(y), "Features and labels have inconsistent lengths."

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Initialize and train the Naive Bayes model
    model = MultinomialNB()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(report)
    
    return accuracy, report

trial1 = train_naive_bayes(df, feature_cols = ['emotion', 'Priority'], target_col = 'eventual')


Accuracy: 0.5696880733944955
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.76      0.16      0.26      6573
    NOTFIXED       0.55      0.95      0.70      7052

    accuracy                           0.57     13625
   macro avg       0.65      0.56      0.48     13625
weighted avg       0.65      0.57      0.49     13625



In [57]:
from sklearn.svm import SVC

def train_svm(df, feature_cols, target_col, test_size=0.2, random_state=42):
    """
    Trains an SVM model on the specified DataFrame using the given feature columns and target column.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    feature_cols (list of str): List of column names to be used as features.
    target_col (str): The name of the target column.
    test_size (float): The proportion of the dataset to include in the test split.
    random_state (int): The seed used by the random number generator.

    Returns:
    accuracy: Accuracy of the model on the test set.
    report: Classification report of the model on the test set.
    """
    if 'label' in target_col:
        df = df[df['Resolution'].str.contains('FIXED', na=False)]

    # Fill missing values in feature columns
    df[feature_cols] = df[feature_cols].fillna(' ')

    # Combine feature columns into a single column for vectorization
    df['combined'] = df[feature_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

    # Encode the combined text column
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['combined'])

    # Encode the target column
    y = df[target_col]

    # Ensure X and y have the same length
    assert X.shape[0] == len(y), "Features and labels have inconsistent lengths."

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Initialize and train the SVM model
    model = SVC()
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(report)
    
    return accuracy, report

trial1 = train_svm(df, feature_cols=['emotion', 'Priority'], target_col='eventual')


Accuracy: 0.5696880733944955
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.76      0.16      0.26      6573
    NOTFIXED       0.55      0.95      0.70      7052

    accuracy                           0.57     13625
   macro avg       0.65      0.56      0.48     13625
weighted avg       0.65      0.57      0.49     13625



In [28]:
trial1 = train_naive_bayes(df, feature_cols = ['emotion', 'Priority'], target_col = 'eventual')

Accuracy: 0.5696880733944955
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.76      0.16      0.26      6573
    NOTFIXED       0.55      0.95      0.70      7052

    accuracy                           0.57     13625
   macro avg       0.65      0.56      0.48     13625
weighted avg       0.65      0.57      0.49     13625



In [27]:
trial2 = train_naive_bayes(df, feature_cols = ['emotion', 'Priority', 'Emotionality'], target_col = 'eventual')

Accuracy: 0.5639633027522936
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.57      0.40      0.47      6573
    NOTFIXED       0.56      0.71      0.63      7052

    accuracy                           0.56     13625
   macro avg       0.56      0.56      0.55     13625
weighted avg       0.56      0.56      0.55     13625



In [29]:
trial2 = train_naive_bayes(df, feature_cols = ['Priority'], target_col = 'eventual')

Accuracy: 0.5696880733944955
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.76      0.16      0.26      6573
    NOTFIXED       0.55      0.95      0.70      7052

    accuracy                           0.57     13625
   macro avg       0.65      0.56      0.48     13625
weighted avg       0.65      0.57      0.49     13625



In [30]:
trial2 = train_naive_bayes(df, feature_cols = ['emotion', 'Emotionality'], target_col = 'eventual')

Accuracy: 0.5069357798165137
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.49      0.49      0.49      6573
    NOTFIXED       0.52      0.52      0.52      7052

    accuracy                           0.51     13625
   macro avg       0.51      0.51      0.51     13625
weighted avg       0.51      0.51      0.51     13625



In [33]:
trial2 = train_naive_bayes(df, feature_cols = ['Description','Priority', 'Emotionality', 'label'], target_col = 'eventual')

Accuracy: 0.5544954128440367
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.52      0.93      0.67      6573
    NOTFIXED       0.76      0.20      0.32      7052

    accuracy                           0.55     13625
   macro avg       0.64      0.57      0.50     13625
weighted avg       0.64      0.55      0.49     13625



In [34]:
trial2 = train_naive_bayes(df, feature_cols = ['Emotionality'], target_col = 'eventual')

Accuracy: 0.5067155963302752
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.49      0.48      0.48      6573
    NOTFIXED       0.52      0.53      0.53      7052

    accuracy                           0.51     13625
   macro avg       0.51      0.51      0.51     13625
weighted avg       0.51      0.51      0.51     13625



In [38]:
trial2 = train_naive_bayes(df, feature_cols = ['emotion', 'Priority', 'Emotionality'], target_col = 'label')

/tmp/ipykernel_149189/2617318525.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature_cols] = df[feature_cols].fillna(' ')
/tmp/ipykernel_149189/2617318525.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined'] = df[feature_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


Accuracy: 0.8303708243947288
Classification Report:
              precision    recall  f1-score   support

        long       0.39      0.03      0.06      1087
       short       0.84      0.99      0.91      5439

    accuracy                           0.83      6526
   macro avg       0.61      0.51      0.48      6526
weighted avg       0.76      0.83      0.77      6526



In [58]:
trial1 = train_svm(df, feature_cols=['Priority'], target_col='eventual')

Accuracy: 0.5696880733944955
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.76      0.16      0.26      6573
    NOTFIXED       0.55      0.95      0.70      7052

    accuracy                           0.57     13625
   macro avg       0.65      0.56      0.48     13625
weighted avg       0.65      0.57      0.49     13625



In [59]:
trial1 = train_svm(df, feature_cols=['emotion'], target_col='Priority')

Accuracy: 0.8557064220183487
Classification Report:
              precision    recall  f1-score   support

          P1       0.00      0.00      0.00       427
          P2       0.00      0.00      0.00       935
          P3       0.86      1.00      0.92     11659
          P4       0.00      0.00      0.00       377
          P5       0.00      0.00      0.00       227

    accuracy                           0.86     13625
   macro avg       0.17      0.20      0.18     13625
weighted avg       0.73      0.86      0.79     13625



/home/spope/anaconda3/envs/reu/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/spope/anaconda3/envs/reu/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/spope/anaconda3/envs/reu/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le